In [ ]:
import numpy as np
import skimage
import skimage.io
import skimage.filters
import skimage.morphology
from scipy import ndimage as ndi
from skimage.filters import threshold_otsu

from skimage.morphology import watershed
from skimage.feature import peak_local_max

import os
import glob

import bebi103

import colorcet

import bokeh
bokeh.io.output_notebook()

import holoviews as hv
hv.extension('bokeh')
bebi103.hv.set_defaults()

First I read the file. I am using one time point, one channel (647).

In [ ]:
# The directory containing the images
data_dir = '../data/barcode'

# glob string for images
im_glob = os.path.join(data_dir, 'Round1_max_composite-t25-c4.tif')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

In [ ]:
# Read data using skimage
im = skimage.io.imread(im_list[0])

In [ ]:
red = im
p = bebi103.image.imshow(red)
bokeh.io.show(p)

I will define a function to adjust the intensity of my image. I will saturate below 0.25% and above 99.75% of all my intensity values.

In [ ]:
# equivalent to matlab imadjust, can set bounds
def imadjust(img, lower_bound=0.25, upper_bound=99.75):
    lower = np.percentile(red, lower_bound)
    upper = np.percentile(red, upper_bound)
    out = (img - lower) * (255 / (upper - lower))
    return np.clip(out, 0, 255, out)

In [ ]:
red_adjust = imadjust(red)

In [ ]:
p = bebi103.image.imshow(red_adjust, saturate_channels=False, min_intensity = 0, max_intensity = 255)
bokeh.io.show(p)

In [ ]:
# convert image to float
red_float = skimage.img_as_float(red_adjust)

I will define two functions to display images side by side.

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.13, 0.13],
    cmap=[None, None],
    colorbar=[False, False]
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
        colorbar=colorbar[0]
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
        colorbar=colorbar[1]
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def show_three_ims(
    im_1,
    im_2,
    im_3,
    titles=[None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13],
    cmap=[None, None, None],
    colorbar=[False, False, False]
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
        colorbar=colorbar[0]
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
        colorbar=colorbar[1]
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
        colorbar=colorbar[2]
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3], ncols=3)

In [ ]:
# Make slice object
zoom1 = np.s_[700:1200, 800:1300]
zoom2 = np.s_[475:775, 150:450]

p = bebi103.image.imshow(red_adjust[zoom1], saturate_channels=False, min_intensity = 0, max_intensity = 255)
bokeh.io.show(p)

## Step 1. Filter

I will apply a LoG filter. This filter detects edges, which are defined by areas of sudden peaks or valleys in the gradient (first derivative) of the pixel intensity values. A peak or a valley in the first derivative means there is a zero-crossing in the second derivative.

In [ ]:
# Compute LoG
# smaller filter = more dots
# around 2 is good for majority of dots, but smaller (~1) might work better for bigger dots?

red_LoG = ndi.filters.gaussian_laplace(red_float, 2)
red_LoG2 = ndi.filters.gaussian_laplace(red_float, 3)

bokeh.io.show(
    show_three_ims(red_adjust, red_LoG, red_LoG2,
    titles=["original", "LoG", "LoG"],
    cmap=[None, colorcet.coolwarm, colorcet.coolwarm],
    colorbar=[False, True, True]))

In [ ]:
# 2x2 square structuring element
selem = skimage.morphology.square(4)

# Do max filter and min filter
red_LoG_max = ndi.filters.maximum_filter(red_LoG, footprint=selem)
red_LoG_min = ndi.filters.minimum_filter(red_LoG, footprint=selem)

# Image of zero-crossings
red_edge = ((red_LoG >= 0) & (red_LoG_min < 0)) | ((red_LoG <= 0) & (red_LoG_max > 0))

# Show result
bokeh.io.show(show_two_ims(red_adjust[zoom1], red_edge[zoom1], titles=["original", "edges"]))

I will now apply a filter to return an image with only zero crossings (the edges of the dots). 

In [ ]:
def zero_crossing_filter(im, thresh):
    """
    Returns image with 1 if there is a zero crossing and 0 otherwise.

    thresh is the the minimal value of the gradient, as computed by Sobel
    filter, at crossing to count as a crossing.
    """
    # Square structuring element
    selem = skimage.morphology.square(3)

    # Do max filter and min filter
    im_max = ndi.filters.maximum_filter(im, footprint=selem)
    im_min = ndi.filters.minimum_filter(im, footprint=selem)

    # Compute gradients using Sobel filter
    im_grad = skimage.filters.sobel(im)

    # Return edges
    return ( (  ((im >= 0) & (im_min < 0))
              | ((im <= 0) & (im_max > 0)))
            & (im_grad >= thresh) )

In [ ]:
# Find zero-crossings
# lower --> more dots, higher --> less dots
# 0.5 is lower bound, 1 is upper bound
# seems like 0.85 is best, 0.9 loses dots, but 0.8 gets background
red_edge_zero = zero_crossing_filter(red_LoG, 0.85)

In [ ]:
# Show result
bokeh.io.show(show_two_ims(red_adjust[zoom1], red_edge_zero[zoom1], titles=["original", "edges"]))

In [ ]:
# Close the edge image
selem = skimage.morphology.disk(1)
red_edge_closed = skimage.morphology.binary_opening(red_edge_zero, selem)

#Show result
bokeh.io.show(show_two_ims(red_edge_zero[zoom1], red_edge_closed[zoom1], titles=["original", "edges"]))


Closing the edge seems to merge dots together, which is bad. I will omit this step from the segmentation process.

Next, I will skeletonize to get single pixel edges.

In [ ]:
# Skeletonize edges
red_edge_sk = skimage.morphology.skeletonize(red_edge_zero)

# See result
bokeh.io.show(show_two_ims(red_adjust[zoom1], red_edge_sk[zoom1], titles=["original", "edges"]))

I will fill the holes created from the edges. Note that this will not fill holes that have any openings in them, even if the openings are very small.

In [ ]:
# Fill holes
red_bw = ndi.morphology.binary_fill_holes(red_edge_sk)

# Show result
bokeh.io.show(show_two_ims(red_adjust[zoom1], red_bw[zoom1], titles=["original", "segmented"]))


I will remove small objects. NOte that this removes any objects that are not filled as well and will remove dots if they are not already filled.

In [ ]:
# Remove small objectes that are not bacteria
red_bw = skimage.morphology.remove_small_objects(red_bw, min_size=40)

# Show result
bokeh.io.show(show_two_ims(red_adjust[zoom1], red_bw[zoom1], titles=["original", "segmented"]))



Finally I will apply my watershed function. This is the same as the one from the nuclear segmentation.

In [ ]:
distance = ndi.distance_transform_edt(red_bw)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((20,20)),
                            labels=red_bw)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=red_bw)
bokeh.io.show(show_three_ims(red_adjust[zoom1], markers[zoom1], labels[zoom1], 
                             titles=['distance', 'local max', 'watershed'], 
                             cmap=[None, colorcet.gray, colorcet.b_glasbey_hv]))

In [ ]:
bokeh.io.show(show_two_ims(red_adjust, labels, 
                             titles=['distance', 'watershed'], 
                             cmap=[None, colorcet.b_glasbey_hv]))

In [ ]:
# Make slice object
zoom2 = np.s_[475:775, 150:450]

bokeh.io.show(show_three_ims(red_adjust[zoom2], labels[zoom2], red_LoG[zoom2], 
                             titles=['distance', 'watershed', 'LoG'], 
                             cmap=[None, colorcet.b_glasbey_hv, colorcet.coolwarm],
                             colorbar=[False, False, True]))

It seems like this code has problems segmenting bigger dots and it seems to have to do with the LoG filter.